### All Imports

In [94]:
# All necessary imports here
import oci
import os.path
import sys
import json
import logging
import pprint
import re
from collections import Counter 
import ipaddr #pip3 install ipaddr

### All Config and Clients

In [95]:
# Read config and create clients (identity,network,etc.)

config = oci.config.from_file()
identity_client = oci.identity.IdentityClient(config)
virtual_network_client = oci.core.VirtualNetworkClient(config)
virtual_network_composite_operations = oci.core.VirtualNetworkClientCompositeOperations(virtual_network_client)

### Logger, Error Handler and Util Methods

In [96]:
# local logger
def local_logger(value):    
    print(value)
    
    
def print_decorator(message):
    print ("====================")
    print (message)
    print ("====================")
    

def error_handle(resource, status, message):
    print ("========{} ERROR============".format(resource))
    print ("{} ERROR".format(status))
    print (message)
    print ("====================")

In [97]:
# Helper methods for extracting and json_lookup
def extract_value_by_field(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [98]:
# helper method to convert response to dictionary
def convert_response_to_dict(oci_response):
    return oci.util.to_dict(oci_response.data)

### JSON Validation Methods 

In [99]:
# Special Characters Regex validation
def special_char_regex_validation(value):
        special_chars_regex = re.compile('[@!#$%^&*()<>?/\|}{~:`]')
        special_char_check = special_chars_regex.search(value)
        if special_char_check is None:            
            return True
        else:
            return False

# IPV4 CIDR Notation Regex Validation        
def ipv4_regex_validation(value):
    ipv4_cidr_regex = re.compile('^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])/(1[0-9]|2[0-9]|3[0-1])$')    
    ipv4_cidr_check = ipv4_cidr_regex.search(value)
    if ipv4_cidr_check is None:
        return False
    else:
        return True
    

## Compartment Handlers

In [100]:
# Check if compartment exists
def check_if_compartment_exist(client, compartment_ocid):
    try:
        compartmentResponse = client.get_compartment(
        compartment_id=compartment_ocid,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        compartment_detail = convert_response_to_dict(compartmentResponse)    
        return True if compartment_detail["id"] == compartment_ocid else False
    except Exception as inst:
        exception = inst
        if inst.status and inst.message:
            error_handle("COMPARTMENT", inst.status, inst.message)
        else:
            error_handle("COMPARTMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False

    
# Check if compartment is active
def check_if_compartment_is_active(client, compartment_ocid):
    try:
        compartment_response = client.get_compartment(
        compartment_id=compartment_ocid, retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        compartment = convert_response_to_dict(compartment_response)
        return True if compartment["lifecycle_state"] == "ACTIVE" else False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("COMPARTMENT", inst.status, inst.message)
        else:
            error_handle("COMPARTMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False
    
# Fetch all compartments by tenancy
def fetch_all_compartments_in_tenancy(client, tenancy_ocid):
    try:
        """Fetch all Compartments in Tenancy , and look across all subtrees."""
        compartmentResponse = oci.pagination.list_call_get_all_results(
            client.list_compartments,
            compartment_id=tenancy_ocid,
            limit=200,
            access_level="ACCESSIBLE",
            compartment_id_in_subtree=True,
            retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        return convert_response_to_dict(compartmentResponse)
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("COMPARTMENT", inst.status, inst.message)
        else:
            error_handle("COMPARTMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Filter compartment by state = active    
def filter_compartments_by_state(compartmentList=[], compartmentState="ACTIVE"):
    try:
        """Filter Compartments by their lifecycle state, ACTIVE| DELETNG | DELETED | CREATING"""
        filteredCompartments = [
            compartment
            for compartment in compartmentList
            if compartment["lifecycle_state"] == compartmentState
        ]
        return filteredCompartments
    except Exception as inst:
            exception = inst        
            if inst.status and inst.message:
                error_handle("COMPARTMENT", inst.status, inst.message)
            else:
                error_handle("COMPARTMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
            return None

# Get compartment OCID from name
def get_compartment_ocid_from_name(client, tenancy_ocid, compartment_name):
    compartment_ocid = None
    try:
        compartments = fetch_all_compartments_in_tenancy(client, tenancy_ocid)
        activeCompartments = filter_compartments_by_state(
            compartmentList=compartments, compartmentState="ACTIVE"
        )
        for compartment in activeCompartments:
            if compartment_name == compartment["name"]:                
                compartment_ocid = compartment["id"]
        return compartment_ocid
    except Exception as inst:
            exception = inst        
            if inst.status and inst.message:
                error_handle("COMPARTMENT", inst.status, inst.message)
            else:
                error_handle("COMPARTMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
            return None



### VCN Handlers

In [101]:
# Check if existing VCN matches by Name
def check_vcn_name_match(client, compartment_id, vcn_name):
    try:
        listVCNReponse = client.list_vcns(compartment_id = compartment_id)
        vcns = convert_response_to_dict(listVCNReponse)
        vcn_names = extract_value_by_field(vcns, "display_name")
        if vcn_name in vcn_names:
            print_decorator("VCN NAME ALREADY EXIST. SKIPPING VCN CREATION")
            return True
        else:
            print_decorator("NO VCN NAME MATCH FOUND. VCN CIDR CHECK IN PROGRESS...")
            return False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False
    
# Check if existing VCN matches by CIDR
def check_vcn_cidr_match(client, compartment_id, vcn_cidr_block):
    is_match_found = False
    try:
        getResponse = client.list_vcns(compartment_id = compartment_id)
        vcns = convert_response_to_dict(getResponse)
        for vcn in vcns:
            vcn_cidr_ip = vcn["cidr_block"].split("/")[0]
            input_cidr_ip = vcn_cidr_block.split("/")[0]
            if vcn["cidr_block"] == vcn_cidr_block or vcn_cidr_ip == input_cidr_ip:                
                is_match_found = True
                break
            else:                                                
                is_match_found = False
        if is_match_found:
            print_decorator("VCN CIDR ALREADY EXIST. SKIPPING VCN CREATION")
        else:
            print_decorator("NO VCN CIDR MATCH FOUND.")            
        return is_match_found
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False

# Check if VCN exist from OCID
def check_vcn_exist_by_ocid(client, compartment_id, vcn_ocid):
    try:
        vcnResponse = client.get_vcn(vcn_id = vcn_ocid)
        vcn = convert_response_to_dict(vcnResponse)
        return True if vcn["id"] == vcn_ocid else False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False

# Check if VCN is is AVAILABLE STATE    
def check_vcn_ocid_is_available(client, compartment_id, vcn_ocid):
    try:
        vcnResponse = client.get_vcn(vcn_id = vcn_ocid)
        vcn = convert_response_to_dict(vcnResponse)
        return True if vcn["lifecycle_state"] == "AVAILABLE" else False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False
    
    
# Get VCN OCID from Name or CIDR
def get_vcn_match_ocid(client, compartment_id, vcn_name = None, vcn_cidr = None):    
    try:
        listVCNReponse = client.list_vcns(compartment_id = compartment_id)
        vcns = convert_response_to_dict(listVCNReponse)
        vcn_ocid = None
        for vcn in vcns:            
            if vcn_name is not None:
                if vcn["display_name"] == vcn_name:
                    vcn_ocid = vcn["id"]
            elif vcn_cidr is not None:
                if vcn["cidr_block"] == vcn_cidr:
                    vcn_ocid = vcn["id"]
        return vcn_ocid
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return False
        

### Check and Create VCN if doesn't exist

In [102]:
# Create VCN
def create_vcn(client, composite_client, vcn):
    try:
        compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], vcn["compartment_name"])
        vcn_details = oci.core.models.CreateVcnDetails(cidr_block=vcn["cidr_block"],
        display_name=vcn["name"],
        compartment_id= compartment_ocid,
        dns_label= vcn["dns_label"])
        VCNResponse = composite_client.create_vcn_and_wait_for_state(
        vcn_details, 
        wait_for_states=[oci.core.models.Vcn.LIFECYCLE_STATE_AVAILABLE]
        )
        vcn = convert_response_to_dict(VCNResponse)
        print_decorator("CREATING VCN")
        return vcn["id"]
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Check and create VCN
def check_create_vcn(client, composite_client, vcn):    
    matched_vcn_ocid = None
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], vcn["compartment_name"])
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    if compartment_exist_check and compartment_available_check:
        print_decorator("COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE")
        vcn_name_check = check_vcn_name_match(client, compartment_ocid, vcn["name"])
        vcn_cidr_check = check_vcn_cidr_match(client, compartment_ocid, vcn["cidr_block"])
        if vcn_name_check:
            matched_vcn_ocid = get_vcn_match_ocid(client, compartment_ocid, vcn["name"])
        if vcn_cidr_check:
            matched_vcn_ocid = get_vcn_match_ocid(client, compartment_ocid, vcn_cidr = vcn["cidr_block"])    
    if matched_vcn_ocid is not None:
        vcn_available_check = check_vcn_ocid_is_available(client, compartment_ocid, matched_vcn_ocid)
        if vcn_available_check:
            return matched_vcn_ocid
    elif matched_vcn_ocid is None:
        return create_vcn(client, composite_client, vcn)


### DHCP Options Handlers

In [103]:
# Check if DHCP Options exists by name based on compartment ID and VCN ID
def check_if_dhcp_options_exists_by_name(client, compartment_id, vcn_ocid, dhcp_name):
    is_match_found = False
    try:
        listDhcpOptions = client.list_dhcp_options(compartment_id = compartment_id,
                                              vcn_id = vcn_ocid,
                                              retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,)
        DhcpOptions = convert_response_to_dict(listDhcpOptions)
        for dhcp in DhcpOptions:
            if dhcp["display_name"] == dhcp_name:
                is_match_found = True
            else:
                is_match_found = False
        return is_match_found
        
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DHCP", inst.status, inst.message)
        else:
            error_handle("DHCP", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None
    
# Check if DHCP Options exists by name based on compartment ID and VCN ID and return dhcp OCID
def get_dhcp_ocid_by_name(client, compartment_id, vcn_ocid, dhcp_name):
    is_match_found_ocid = None
    try:
        listDhcpOptions = client.list_dhcp_options(compartment_id = compartment_id,
                                              vcn_id = vcn_ocid,
                                              retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,)
        DhcpOptions = convert_response_to_dict(listDhcpOptions)
        for dhcp in DhcpOptions:
            if dhcp["display_name"] == dhcp_name:
                is_match_found_ocid = dhcp["id"]
            else:
                is_match_found_ocid = None
        return is_match_found_ocid
        
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DHCP", inst.status, inst.message)
        else:
            error_handle("DHCP", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Check DHCP Options if exists by OCID based on compartment ID and VCN ID
def check_if_dhcp_options_exists_by_ocid(client, compartment_id, vcn_ocid, dhcp_ocid):
    is_match_found = False
    try:
        listDhcpOptions = client.list_dhcp_options(compartment_id = compartment_id,
                                              vcn_id = vcn_ocid,
                                              retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,)
        DhcpOptions = convert_response_to_dict(listDhcpOptions)
        for dhcp in DhcpOptions:
            if dhcp["id"] == dhcp_ocid:
                print(dhcp)
                is_match_found = True
            else:
                is_match_found = False
        return is_match_found
        
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DHCP", inst.status, inst.message)
        else:
            error_handle("DHCP", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None 
    

    
# Check DHCP Options if exists by name and state is available based on compartment ID and VCN ID
def check_if_dhcp_options_lifecycle_ocid(client, compartment_id, vcn_ocid, dhcp_ocid):
    is_match_found = False
    try:
        listDhcpOptions = client.list_dhcp_options(compartment_id = compartment_id,
                                              vcn_id = vcn_ocid,
                                              retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,)
        DhcpOptions = convert_response_to_dict(listDhcpOptions)
        for dhcp in DhcpOptions:
            if dhcp["lifecycle_state"] == "AVAILABLE":
                is_match_found = True
            else:
                is_match_found = False
        return is_match_found
        
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DHCP", inst.status, inst.message)
        else:
            error_handle("DHCP", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None 

### Check and Create DHCP Options if doesn't exist

In [104]:
# Create DHCP
"""server_type can be "VcnLocal", "VcnLocalPlusInternet", "CustomDnsServer", "UNKNOWN_ENUM_VALUE"
type can be "DomainNameServer" or "SearchDomain"
"""
def create_dhcp(client, compartment_id, vcn_ocid, dhcp_options, dhcp_name):   
    try:
        dhcpDetails = oci.core.models.CreateDhcpDetails(compartment_id = compartment_id, options = dhcp_options, vcn_id = vcn_ocid, display_name=dhcp_name)
        dhcpResponse = client.create_dhcp_options(create_dhcp_details= dhcpDetails)
        dhcpRes = convert_response_to_dict(dhcpResponse)
        return dhcpRes
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("VCN", inst.status, inst.message)
        else:
            error_handle("VCN", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None 

    
def check_create_dhcp(client, compartment_name, vcn, vcn_ocid, dhcp):
    matched_dhcp_ocid = None 
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], compartment_name)
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    
    vcn_exist = check_vcn_exist_by_ocid(client, compartment_ocid, vcn_ocid)
    vcn_available_state = check_vcn_ocid_is_available(client, compartment_ocid, vcn_ocid)
    
    dhcp_name_check = check_if_dhcp_options_exists_by_name(client, compartment_ocid, vcn_ocid, dhcp["name"])

    if compartment_exist_check and compartment_available_check:
        print_decorator("COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE")
        if vcn_exist and vcn_available_state and dhcp_name_check:
            matched_dhcp_ocid = get_dhcp_ocid_by_name(client, compartment_ocid, vcn_ocid, dhcp["name"])        
            dhcp_available_state = check_if_dhcp_options_lifecycle_ocid(client, compartment_ocid, vcn_ocid, matched_dhcp_ocid)
            if matched_dhcp_ocid is not None and dhcp_available_state:
                print_decorator("DHCP ALREADY EXIST")
                return matched_dhcp_ocid
        elif matched_dhcp_ocid is None:
            matched_dhcp_ocid = create_dhcp(virtual_network_client, compartment_ocid, vcn_ocid, dhcp["options"], dhcp["name"])
    return matched_dhcp_ocid


### DRG Handlers

In [105]:
# Check if DRG exist by name
def check_if_drg_exist_by_name(client, compartment_ocid, drg_name):
    try:
        listDRGs = client.list_drgs(compartment_id = compartment_ocid)
        drgs = convert_response_to_dict(listDRGs)
        drg_name_extract = extract_value_by_field(drgs, "display_name")
        return True if drg_name in drg_name_extract else False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG", inst.status, inst.message)
        else:
            error_handle("DRG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None
    

# Check if DRG exist already by OCID and State in Compartment        
def check_drg_ocid_is_available(client, compartment_ocid, drg_ocid):
    try:
        drg = client.get_drg(drg_id = drg_ocid, retry_strategy= oci.retry.DEFAULT_RETRY_STRATEGY)
        drg_dict = convert_response_to_dict(drg)
        if drg_dict is not None and drg_dict["lifecycle_state"] == "AVAILABLE":
            return True
        else:
            return False 
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG", inst.status, inst.message)
        else:
            error_handle("DRG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None
    

# Get DRG OCID from DRG Name
def get_drg_match_ocid(client, compartment_ocid, drg_name):
    try:
        listDRGs = client.list_drgs(compartment_id = compartment_ocid)
        drgs = convert_response_to_dict(listDRGs)
        for drg in drgs:
            if drg["display_name"] == drg_name:
                return drg["id"]
            else:
                return None
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG", inst.status, inst.message)
        else:
            error_handle("DRG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

### Check and create DRG if doesn't exist

In [106]:
# Create DRG
def create_drg(client, drg):
    try:
        compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], drg["compartment_name"])
        drg_result = client.create_drg(
            oci.core.models.CreateDrgDetails(
                compartment_id=compartment_ocid,
                display_name= drg["name"]
            )
        )
        drg = oci.wait_until(
            client,
            client.get_drg(drg_result.data.id),
            'lifecycle_state',
            'AVAILABLE'
        )
        print_decorator("CREATING DRG")
        drg_new = convert_response_to_dict(drg)
        return drg_new["id"]
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG", inst.status, inst.message)
        else:
            error_handle("DRG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Check and create DRG if doesn't exist
def check_create_drg(client, drg):
    matched_drg_ocid = None
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], drg["compartment_name"])
    drg_name_check = check_if_drg_exist_by_name(client, compartment_ocid, drg["name"])    
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    if compartment_exist_check and compartment_available_check:
        print_decorator("COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE")
        if drg_name_check:
            matched_drg_ocid = get_drg_match_ocid(client, compartment_ocid, drg["name"])        
            drg_available_state = check_drg_ocid_is_available(client, compartment_ocid, matched_drg_ocid)
            if matched_drg_ocid is not None and drg_available_state:
                print_decorator("DRG ALREADY EXIST")
                return matched_drg_ocid
        elif matched_drg_ocid is None:
            matched_drg_ocid = create_drg(client, drg)
    return matched_drg_ocid
                    

### DRG Attachment Handlers

In [107]:
# Check DRG attachment status 
def get_drg_attachment_status(client, drg_attachment_ocid):
    try:
        drg_attachment = client.get_drg_attachment(drg_attachment_id = drg_attachment_ocid)
        drg_attachment_dict = convert_response_to_dict(drg_attachment)
        if drg_attachment_dict is not None and drg_attachment_dict["lifecycle_state"]=="ATTACHED":
            return True
        else:
            return False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG ATTACHMENT", inst.status, inst.message)
        else:
            error_handle("DRG ATTACHMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None


# Get DRG attachment ocid from DRG OCID
def filter_drg_attachment_id(client,compartment_ocid, drg_id):
    try:
        drg_attachment_id = None
        drg_attachments = client.list_drg_attachments(compartment_id = compartment_ocid)
        drg_attachments_dict = convert_response_to_dict(drg_attachments)
        for drg_attachment in drg_attachments_dict:
            if drg_attachment["drg_id"] == drg_id:
                drg_attachment_id =  drg_attachment["id"]            
            else:
                drg_attachment_id = None
        return drg_attachment_id
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG ATTACHMENT", inst.status, inst.message)
        else:
            error_handle("DRG ATTACHMENT", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

    

### Check and create DRG attachment if doesn't exist

In [108]:
# Attach newly created DRG to VCN using VCN OCID
def drg_attach(client, vcn_ocid, drg_ocid, drg_name):
    try:
        drg_attach_result = client.create_drg_attachment(
            oci.core.models.CreateDrgAttachmentDetails(
                display_name= drg_name,
                vcn_id=vcn_ocid,
                drg_id=drg_ocid
            )
        )
        drg_attachment = oci.wait_until(
            client,
            client.get_drg_attachment(drg_attach_result.data.id),
            'lifecycle_state',
            'ATTACHED'
        )
        print_decorator('CREATED DRG ATTACHMENT')
        drg_attach = convert_response_to_dict(drg_attachment)
        return drg_attach["id"]
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("DRG", inst.status, inst.message)
        else:
            error_handle("DRG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Check and create DRG if doesn't exist
def check_create_drg_attachment(client, drg, vcn, drg_ocid, vcn_ocid):
    matched_drg_attachment_ocid = None    
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], drg["compartment_name"])
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    
    vcn_exist = check_vcn_exist_by_ocid(client, compartment_ocid, vcn_ocid)
    vcn_available_state = check_vcn_ocid_is_available(client, compartment_ocid, vcn_ocid)
    
    drg_name_check = check_if_drg_exist_by_name(client, compartment_ocid, drg["name"])
    if compartment_exist_check and compartment_available_check:
        print_decorator("COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE")
        if vcn_exist and vcn_available_state and drg_name_check:
            drg_ocid = get_drg_match_ocid(client, compartment_ocid, drg["name"])        
            drg_available_state = check_drg_ocid_is_available(client, compartment_ocid, drg_ocid)              
            if drg_ocid is not None and drg_available_state:
                drg_attachment_ocid = filter_drg_attachment_id(client, compartment_ocid, drg_ocid)
                if drg_attachment_ocid is not None:
                    drg_attached_state = get_drg_attachment_status(client, drg_attachment_ocid)
                    if drg_attachment_ocid is not None and drg_attached_state:                                                    
                        matched_drg_attachment_ocid = drg_attachment_ocid
                        print_decorator("DRG ALREADY ATTACHED TO EXISTING VCN")
                        return matched_drg_attachment_ocid 
                elif matched_drg_attachment_ocid is None:
                    matched_drg_attachment_ocid = drg_attach(client, vcn_ocid, drg_ocid, drg["name"])
    return matched_drg_attachment_ocid

### LPG Handlers

In [109]:
# Check if LPG name matches
def check_if_lpg_exist_by_name(client, compartment_ocid, vcn_ocid, lpg_name):
    if_match_found = False
    try:
        listLpgs = client.list_local_peering_gateways(compartment_id = compartment_ocid, vcn_id= vcn_ocid)
        lpgs = convert_response_to_dict(listLpgs)
        lpg_names = extract_value_by_field(lpgs, "display_name")
        if lpg_name in lpg_names:            
            if_match_found = True
        else:
            if_match_found = False
        return if_match_found
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("LPG", inst.status, inst.message)
        else:
            error_handle("LPG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None


# Get LPG OCID from matching name
def get_lpg_ocid_by_lpg_name (client, compartment_ocid, vcn_ocid, lpg_name):
    lpg_id= None
    try:
        listLpgs = client.list_local_peering_gateways(compartment_id = compartment_ocid, vcn_id= vcn_ocid)
        lpgs = convert_response_to_dict(listLpgs)
        for lpg in lpgs:
            if lpg["display_name"] == lpg_name:
                lpg_id = lpg["id"]           
        return lpg_id
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("LPG", inst.status, inst.message)
        else:
            error_handle("LPG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Check for LPG state
def check_if_lpg_is_available(client, lpg_ocid, compartment_ocid):
    is_match_found = False
    try:
        lpg = client.get_local_peering_gateway(local_peering_gateway_id = lpg_ocid)
        lpg_dict = convert_response_to_dict(lpg)
        if lpg_dict["lifecycle_state"] == "AVAILABLE":
            is_match_found = True
        else:
            is_match_found = False
        return is_match_found
    except Exception as inst:
        print(inst)
        exception = inst        
        if inst.status and inst.message:
            error_handle("LPG", inst.status, inst.message)
        else:
            error_handle("LPG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

# Check if connection is established
def check_peering_status(client, lpg_ocid):
    try:
        lpg = client.get_local_peering_gateway(local_peering_gateway_id = lpg_ocid,
                                              retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,)
        lpg_dict = convert_response_to_dict(lpg)
        if lpg_dict["peering_status"] == "PEERED":
            return True
        else:
            return False
    except Exception as inst:
        exception = inst        
        if inst.status and inst.message:
            error_handle("LPG", inst.status, inst.message)
        else:
            error_handle("LPG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None    

### Check and create LPG if doesn't exist

In [110]:
# Create Local Peering Gateway been Hub and Spokes VCN
def create_local_peering_gateway(composite_client, lpg_name, compartment_id, vcn_ocid):
    try:
        create_lpg_details = oci.core.models.CreateLocalPeeringGatewayDetails(compartment_id = compartment_id, display_name = lpg_name, vcn_id = vcn_ocid)
        create_lpg_response = composite_client.create_local_peering_gateway_and_wait_for_state(
        create_lpg_details,
        wait_for_states=[oci.core.models.LocalPeeringGateway.LIFECYCLE_STATE_AVAILABLE]
            )
        lpg = create_lpg_response
        lpg_dict = convert_response_to_dict(lpg)
        print_decorator("CREATED LPG")
        return lpg_dict["id"]
    
    except Exception as inst:
        if inst.status and inst.message:
            error_handle("LPG", inst.status, inst.message)
        else:
            error_handle("LPG", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None

def check_create_lpg(client, composite_client, vcn, lpg, vcn_ocid):    
    matched_lpg_ocid = None    
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], lpg["compartment_name"])
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    
    vcn_exist = check_vcn_exist_by_ocid(client, compartment_ocid, vcn_ocid)
    vcn_available_state = check_vcn_ocid_is_available(client, compartment_ocid, vcn_ocid)
    
    lpg_name_check = check_if_lpg_exist_by_name(client, compartment_ocid, vcn_ocid, lpg["name"])

    if compartment_exist_check and compartment_available_check:
        print_decorator("COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE")
        if vcn_exist and vcn_available_state and lpg_name_check:
            matched_lpg_ocid = get_lpg_ocid_by_lpg_name(client, compartment_ocid, vcn_ocid, lpg["name"])        
            lpg_available_state = check_if_lpg_is_available(client, matched_lpg_ocid, compartment_ocid)
            if matched_lpg_ocid is not None and lpg_available_state:
                print_decorator("LPG ALREADY EXIST")
                return matched_lpg_ocid
        elif matched_lpg_ocid is None:
            matched_lpg_ocid = create_local_peering_gateway(composite_client, lpg["name"], compartment_ocid, vcn_ocid)
    return matched_lpg_ocid



### LPG Connection Establishment

In [111]:
# Connect Local Peering Gateway between Hub and Spokes VCN
def connect_local_peering_gateway(client, source_lpg_id, peer_lpg_id):
    try:
        connect_lpg_details = oci.core.models.ConnectLocalPeeringGatewaysDetails(peer_id = peer_lpg_id)
        connect_lpg = client.connect_local_peering_gateways(source_lpg_id, connect_local_peering_gateways_details = connect_lpg_details)
        print_decorator('LPG CONNECTED')
        return convert_response_to_dict(connect_lpg)
    except Exception as inst:
        if inst.status and inst.message:
            error_handle("LPG CONNECTION", inst.status, inst.message)
        else:
            error_handle("LPG CONNECTION", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None
    
# Check and create LPG connection
def check_connect_lpg(client, lpg, source_lpg_ocid, peer_lpg_ocid):
    if_source_lpg_peered = False
    if_target_lpg_peered = False
    
    peering_response = None
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], lpg["compartment_name"])
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    
#     vcn_exist = check_vcn_exist_by_ocid(client, lpg["compartment_id"], source_vcn_ocid)
#     vcn_available_state = check_vcn_ocid_is_available(client, lpg["compartment_id"], source_vcn_ocid)
    
    
    if_source_lpg_peered = check_peering_status(client, source_lpg_ocid)
    if_target_lpg_peered = check_peering_status(client, peer_lpg_ocid)
    
#     if vcn_exist and vcn_available_state:
    if if_source_lpg_peered:
        print_decorator("SOURCE LPG {} IS ALREADY PEERED".format(source_lpg_ocid))
        peering_response = None
    elif if_target_lpg_peered:
        print_decorator("TARGET LPG {} IS ALREADY PEERED".format(peer_lpg_ocid))
        peering_response = None
    elif not if_source_lpg_peered and not if_target_lpg_peered:
        peering_response = connect_local_peering_gateway(client, source_lpg_ocid, peer_lpg_ocid)
    else:
        peering_response = None
    return peering_response


### Subnet Handlers

In [112]:
# Match subnet name from list
def check_subnet_name_match(client, compartment_ocid, subnet_detail, vcn_ocid):
    is_match_found = False
    try:
        subnetsList = client.list_subnets(
        compartment_id = compartment_ocid,
        vcn_id = vcn_ocid,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        subnets = convert_response_to_dict(subnetsList)
        for subnet in subnets:
            if subnet["display_name"] == subnet_detail["name"]:
                is_match_found = True
        return is_match_found
    
    except Exception as inst:
        if inst.status and inst.message:
            error_handle("SUBNET", inst.status, inst.message)
        else:
            error_handle("SUBNET", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None
    
# Check if subnet is available lifecycle_state from subnet name
def check_subnet_availability(client,compartment_ocid, subnet_detail, vcn_ocid):
    is_match_found = False
    try:
        subnetsList = client.list_subnets(
        compartment_id = compartment_ocid,
        vcn_id = vcn_ocid,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        subnets = convert_response_to_dict(subnetsList)
        for subnet in subnets:
            if subnet["display_name"] == subnet_detail["name"]:                
                if subnet["lifecycle_state"] == "AVAILABLE":
                    is_match_found = True
        return is_match_found
    
    except Exception as inst:
        if inst.status and inst.message:
            error_handle("SUBNET", inst.status, inst.message)
        else:
            error_handle("SUBNET", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None


# Check and return subnet ocid if available by name
def get_subnet_ocid_by_name(client, compartment_ocid, subnet_detail, vcn_ocid):
    matched_subnet_ocid = None
    try:
        subnetsList = client.list_subnets(
        compartment_id = compartment_ocid,
        vcn_id = vcn_ocid,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        subnets = convert_response_to_dict(subnetsList)
        for subnet in subnets:
            if subnet["display_name"] == subnet_detail["name"]:
                matched_subnet_ocid = subnet["id"]
        return matched_subnet_ocid
    
    except Exception as inst:
        if inst.status and inst.message:
            error_handle("SUBNET", inst.status, inst.message)
        else:
            error_handle("SUBNET", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None           

In [113]:
# Create Subnet
def create_subnet(client,compartment_ocid, subnet_detail, vcn_ocid):
    assign_public_ip = True if subnet_detail["is_public"]=="True" else False
    try:
        create_subnet_details = oci.core.models.CreateSubnetDetails(
        cidr_block = subnet_detail["cidr"],display_name = subnet_detail["name"],
            compartment_id = compartment_ocid,
            prohibit_public_ip_on_vnic = assign_public_ip,
            vcn_id = vcn_ocid)
        subnet_response = client.create_subnet(
            create_subnet_details = create_subnet_details,
        )                
        subnet = convert_response_to_dict(subnet_response)
        print_decorator("SUBNET CREATED")
        return subnet["id"]
    
    except Exception as inst:
        if inst.status and inst.message:
            error_handle("SUBNET", inst.status, inst.message)
        else:
            error_handle("SUBNET", "UNKNOWN", "UNKNOWN ERROR MESSAGE")
        return None
    
def check_create_subnet(client, subnet_details, vcn_ocid):
    matched_subnet_ocid = None
    compartment_ocid = get_compartment_ocid_from_name(identity_client, config["tenancy"], subnet_details["compartment_name"])
    compartment_exist_check = check_if_compartment_exist(identity_client, compartment_ocid)        
    compartment_available_check = check_if_compartment_is_active(identity_client, compartment_ocid)
    
    if compartment_exist_check and compartment_available_check:        
        vcn_exist = check_vcn_exist_by_ocid(client, compartment_ocid, vcn_ocid)
        vcn_available_state = check_vcn_ocid_is_available(client, compartment_ocid, vcn_ocid)
        
        if vcn_exist and vcn_available_state:
            subnet_name_match = check_subnet_name_match(client,compartment_ocid, subnet_details, vcn_ocid)
            if subnet_name_match:
                subnet_availability = check_subnet_availability(client,compartment_ocid, subnet_details, vcn_ocid)
                if subnet_availability:
                    subnet_ocid = get_subnet_ocid_by_name(client, compartment_ocid, subnet_details, vcn_ocid)
                    matched_subnet_ocid = subnet_ocid
                    print_decorator("SUBNET ALREADY EXISTS. SKIPPING SUBNET CREATION")
            if matched_subnet_ocid is None:
                matched_subnet_ocid = create_subnet(client, subnet_details, vcn_ocid)
    return matched_subnet_ocid
    
    

### JSON sanity check method

In [114]:
# File exists
def file_exist_check(filePath):
    try:
        does_exist = os.path.exists(filePath)
        if does_exist:
            print_decorator("FILE EXISTS IN DIRECTORY!\nPROCEEDING FOR SANITY CHECK !")
            return True
        else:
            print_decorator("MISSING FILE.\nFILE DOES NOT EXIST IN DIRECTORY !")
            return False
    except Exception as inst:
        print_decorator(inst)
        return False


# Sanity check
def json_sanity_check(payload):
    completed = False
    all_names = []
    cidr_blocks = []
    
    hub = payload["hub"]
    hub_vcn = hub["vcn"]
    spokes = payload["spokes"]
    spoke_vcn = []
    
    for spoke in spokes:
        spoke_vcn.append(spoke["vcn"])
    
    all_names.extend(extract_value_by_field(payload, "name"))
    all_names.extend(extract_value_by_field(payload, "dns_label"))
    
    cidr_blocks.extend(extract_value_by_field(hub, "cidr_block"))
    cidr_blocks.extend(extract_value_by_field(spoke_vcn, "cidr_block"))

    is_cidr_sanity_check = True
    is_cidr_overlap_check = False
    is_names_sanity_check = True
    
    for name in all_names:
        is_names_sanity_check=special_char_regex_validation(name)
        if not is_names_sanity_check:
            break

    for index_out, cidr_out in enumerate(cidr_blocks):
        is_cidr_sanity_check = ipv4_regex_validation(cidr_out)
        
        if not is_cidr_sanity_check:
            break
            
        for index_in, cidr_in in enumerate(cidr_blocks):                
                if index_in != index_out:                                    
                    ip_range_out = ipaddr.IPNetwork(cidr_out)
                    ip_range_in = ipaddr.IPNetwork(cidr_in)
                    is_cidr_overlap_check = ip_range_out.overlaps(ip_range_in)
                    if is_cidr_overlap_check:
                        break
       
    if not is_cidr_sanity_check:
        print_decorator("INVALID VCN CIDR BLOCKS.\nERROR IN CIDR BLOCK FOUND")
        completed = False
    
    if is_cidr_overlap_check:
        print_decorator("INVALID VCN CIDR BLOCKS.\nCIDR BLOCKS ARE OVERLAPPING")
        completed = False
                                              
    if not is_names_sanity_check:
        print_decorator("IMPROPER NAMING CONVENTION.\nSPECIAL CHARACTERS IN NAMES NOT ALLOWED")
        completed = False
    
    if is_cidr_sanity_check and not is_cidr_overlap_check and is_names_sanity_check:
        print_decorator("FILE SANITY CHECK COMPLETED SUCCESSFULLY\nNO ERROR FOUND")
        completed = True
    return completed

### Read file - Sanity Check - Create VCN - Create DRG - Attach DRG - Create LPG - Connect LPG

In [115]:
def read_file(filePath):
    file_exists = file_exist_check(filePath)
    if file_exists:
         with open(filePath) as hub_spokes:
            payload = json.load(hub_spokes)
            sanity_check = json_sanity_check(payload)
            if sanity_check:
                hub = payload["hub"]
                hub_vcn = hub["vcn"]
                spokes = payload["spokes"]
                spoke_vcns = []
                spoke_vcn_ocids = []

                for spoke in spokes:
                    spoke_vcns.append(spoke["vcn"])
                
                """CREATE HUB VCN"""
                hub_vcn_ocid = check_create_vcn(virtual_network_client, virtual_network_composite_operations, hub_vcn)                
                
                
                """CREATE SPOKE VCNS"""
                for index,spoke_vcn in enumerate(spoke_vcns):
                    spoke_vcn_ocid = check_create_vcn(virtual_network_client, virtual_network_composite_operations, spoke_vcn)
                    spoke_vcn_ocids.append(spoke_vcn_ocid)
                    
                """CREATE HUB DRG"""
                hub_drg_ocid = check_create_drg(virtual_network_client, hub_vcn["drg"])
                
                """CREATE HUB DRG ATTACHMENT"""
                hub_drg_attachment_ocid = check_create_drg_attachment(virtual_network_client, hub_vcn["drg"], hub_vcn, hub_drg_ocid, hub_vcn_ocid)
                
                """CREATE HUB LPGs"""
                hub_lpg_ocids = []
                spoke_lpg_ocids = []                
                
                hub_lpg_ocids.append(check_create_lpg(virtual_network_client, virtual_network_composite_operations, hub_vcn, hub_vcn["lpg"], hub_vcn_ocid))
                
                for spoke_vcn in spoke_vcns:
                    hub_lpg_ocids.append(check_create_lpg(virtual_network_client, virtual_network_composite_operations, hub_vcn, spoke_vcn["lpg"], hub_vcn_ocid))
                
                """CREATE HUB AND SPOKE LPGs"""
                for index, spoke_vcn_ocid in enumerate(spoke_vcn_ocids):
                    spokes_list = list(spokes)
                    spoke_lpg_ocids.append(check_create_lpg(virtual_network_client, virtual_network_composite_operations, spokes_list[index]["vcn"], spokes_list[index]["vcn"]["lpg"], spoke_vcn_ocid))
                    
                """ATTACH HUB AND SPOKE VCN THROUGH LPGs"""
                for hub, spoke in zip( spoke_lpg_ocids, hub_lpg_ocids):
                    check_connect_lpg(virtual_network_client, hub_vcn["lpg"], hub, spoke)
                
                """CREATE SUBNET FOR HUB"""
                hub_subnet_ocids = []
                hub_subnet_dhcp_ocids = []
                for subnet in hub_vcn["subnets"]:
                    subnet_ocid = check_create_subnet(virtual_network_client, subnet, hub_vcn_ocid)  
                    hub_subnet_ocids.append(subnet_ocid)
                    hub_subnet_dhcp_ocid = check_create_dhcp(virtual_network_client, subnet["compartment_name"], hub_vcn, hub_vcn_ocid, subnet["dhcp"])
                    hub_subnet_dhcp_ocids.append(hub_subnet_dhcp_ocid)
                
                """CREATE SUBNET FOR SPOKES"""
                spoke_subnet = []
                spoke_subnet_dhcp_ocids = []
                for spoke_vcn_ocid,spoke_vcn in zip(spoke_vcn_ocids, spoke_vcns):                    
                    for subnet in spoke_vcn["subnets"]:
                        subnet_ocid = check_create_subnet(virtual_network_client, subnet, spoke_vcn_ocid)                        
                        spoke_subnet.append({spoke_vcn["name"]: subnet})
                        spoke_subnet_dhcp_ocid = check_create_dhcp(virtual_network_client, subnet["compartment_name"], spoke_vcn, spoke_vcn_ocid, subnet["dhcp"])
                        spoke_subnet_dhcp_ocids.append(spoke_subnet_dhcp_ocid)
    

In [116]:
read_file('./hub_spokes.json')

FILE EXISTS IN DIRECTORY!
PROCEEDING FOR SANITY CHECK !
FILE SANITY CHECK COMPLETED SUCCESSFULLY
NO ERROR FOUND
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
VCN NAME ALREADY EXIST. SKIPPING VCN CREATION
VCN CIDR ALREADY EXIST. SKIPPING VCN CREATION
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
VCN NAME ALREADY EXIST. SKIPPING VCN CREATION
VCN CIDR ALREADY EXIST. SKIPPING VCN CREATION
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
VCN NAME ALREADY EXIST. SKIPPING VCN CREATION
VCN CIDR ALREADY EXIST. SKIPPING VCN CREATION
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
DRG ALREADY EXIST
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
DRG ALREADY ATTACHED TO EXISTING VCN
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
LPG ALREADY EXIST
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
LPG ALREADY EXIST
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
LPG ALREADY EXIST
COMPARTMENT EXIST AND IS IN AVAILABLE LIFECYCLE STATE
LPG ALREADY EXIST

In [ ]:
vcn_ocids = [
    'ocid1.vcn.oc1.phx.amaaaaaa43cggciacicznvdwo3ot6cxgb7tt7edy3g332cpajvcykptuinlq',
    'ocid1.vcn.oc1.phx.amaaaaaa43cggciaiukfhfkegzzcq4sk5ybyfg6rh6ecs5yb7pvtqf7qvvjq',
]

for vcn_ocid in vcn_ocids:
    response = virtual_network_client.delete_vcn(vcn_ocid)
    result = convert_response_to_dict(response)
    print(result)
    